# initialization

In [1]:
import os

In [2]:
%env PYSPARK_SUBMIT_ARGS=--driver-java-options -Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell

env: PYSPARK_SUBMIT_ARGS=--driver-java-options -Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell


In [3]:
",".join(["/opt/gluten/jars/"+l for l in os.listdir("/opt/gluten/jars/")])

'/opt/gluten/jars/gluten-velox-bundle.jar'

In [4]:
# Local path to gluten jar.
gluten_target_jar=",".join(["/opt/gluten/jars/"+l for l in os.listdir("/opt/gluten/jars/")])

# Select workload. Can be either 'tpch' or 'tpcds'.
workload='tpch'

# Run with gluten. If False, run Spark.
run_gluten=True

# TPC tables
#tpch_tabledir='/opt/spark/database/tpch_sf10_parquet_zstd'
#tpcds_tabledir='/opt/spark/database/tpch_sf10_parquet_zstd'

tabledir = 's3a://presto-workload/tpch_sf100_parquet_zstd/'

# TPC queries
#tpch_query_path='/opt/spark/tpch-queries'
#tpcds_query_path='/opt/spark/tpcds-queries'

tpc_query_path = '/opt/spark/tpch-queries/'

# Parallelism
executors_per_node=1

#gluten_tpch_task_per_core=2
#gluten_tpcds_task_per_core=4
#spark_tpch_task_per_core=8
#spark_tpcds_task_per_core=8

task_per_core=2

# Offheap ratio. 0 to disable offheap for Spark.
# onheap:offheap = 1:2
#spark_offheap_ratio=2.0
# onheap:offheap = 1:7
#gluten_offheap_ratio=7.0

offheap_ratio = 7.0

# spark.io.compression.codec
spark_codec='lz4'
# spark.gluten.sql.columnar.shuffle.codec
gluten_codec='lz4'

In [5]:
%run ./native_sql_initialize.ipynb

home: /home/spark
cwd: /opt/spark/work-dir/ipython


,Environment Variable,Value
0,HOSTNAME,ip-10-167-102-54.ec2.internal
1,SHARED_LIBS,/shared-libs
2,JAVA_HOME,/usr/lib/jvm/java-17-openjdk
3,VCPKG_BINARY_SOURCES,"clear;files,/var/cache/vcpkg,readwrite"
4,PWD,/opt/spark/work-dir
5,HOME,/home/spark
6,CCACHE_DIR,/root/.cache/ccache
7,GCC_TOOL,/opt/rh/gcc-toolset-11
8,SPARK_HOME,/opt/spark
9,TERM,xterm-color


localhost: ip-10-167-102-54.ec2.internal
ip: 10.167.102.54
Spark version from SPARK_HOME: 3.5.2


# Application Level Configuration

In [6]:
if run_gluten:
    sct=GlutenSparkContext(executors_per_node, task_per_core, gluten_target_jar, offheap_ratio)
    sct.conf.set('spark.gluten.sql.columnar.shuffle.codec', gluten_codec)
else:
    sct=VanillaSparkContext(executors_per_node, task_per_core, gluten_target_jar, offheap_ratio)
    sct.conf.set('spark.io.compression.codec', spark_codec)

#for s3 access
sct.conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
sct.conf.set('spark.hadoop.fs.s3a.access.key', '')
sct.conf.set('spark.hadoop.fs.s3a.secret.key', '')
sct.conf.set('spark.hadoop.fs.s3a.endpoint', '')



In [7]:
if workload.lower()=="tpch":
    bm=TPCHBenchmark(sct, tabledir, 'parquet', tpc_query_path)
else:
    bm=TPCDSBenchmark(sct, tabledir, 'parquet', tpc_query_path)
bm.initialize()


            executors per node: 1
            parallelism: 16
            executor memory: 1704m
            offheap memory: 11932m
        
spark.serializer:  org.apache.spark.serializer.KryoSerializer
master:  spark://ip-10-167-102-54.ec2.internal:7077


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/17 05:32:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/17 05:32:41 WARN VeloxListenerApi: Memory overhead is set to 2048MiB which is smaller than the recommended size 3579MiB. This may cause OOM.
W20250517 05:32:42.509517  1575 MemoryArbitrator.cpp:84] Query memory capacity[1.50GB] is set for NOOP arbitrator which has no capacity enforcement
/opt/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


appid:  app-20250517053242-0003
SparkConf:


,key,value
0,spark.eventLog.enabled,true
1,spark.executor.memoryOverhead,2048m
2,spark.master,spark://ip-10-167-102-54.ec2.internal:7077
3,spark.sql.catalog.iceberg.uri,thrift://a74b6e20f132e4eac840a591d5caf0ba-c3bc190f0aee9cb5.elb.us-east-1.amazonaws.com:9083
4,spark.hadoop.fs.s3a.aws.credentials.provider,org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider
5,spark.executor.extraClassPath,/opt/gluten/jars/gluten-velox-bundle.jar
6,spark.sql.catalog.spark_catalog,org.apache.iceberg.spark.SparkSessionCatalog
7,spark.driver.extraClassPath,/opt/gluten/jars/gluten-velox-bundle.jar
8,spark.hadoop.fs.s3a.endpoint,https://s3.us-east-1.amazonaws.com
9,spark.gluten.sql.columnar.shuffle.codec,lz4


start run:  app-20250517053242-0003
http://10.167.102.54:18080/history/app-20250517053242-0003/jobs/


# Run Workload

In [9]:
bm.test_tpc.power_run(explain=False, print_result=False, load_table=True, action=lambda df: df.collect())

Loading all tables: ['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier']


start query q1, application id app-20250517053242-0003
Start time : 16331.875452369


start query q2, application id app-20250517053242-0003
Start time : 16359.563025608


start query q3, application id app-20250517053242-0003
Start time : 16377.281476317


start query q4, application id app-20250517053242-0003
Start time : 16409.977656464


...

start query q18, application id app-20250517053242-0003
Start time : 17077.526507092


start query q19, application id app-20250517053242-0003
Start time : 17197.64954849


start query q20, application id app-20250517053242-0003
Start time : 17210.376327304


start query q21, application id app-20250517053242-0003
Start time : 17231.146678051


start query q22, application id app-20250517053242-0003
Start time : 17523.552843788


In [10]:
bm.test_tpc.print_result()

{'q1': {'query_name': 'q1', 'application_id': 'app-20250517053242-0003', 'application_time_taken': 27.58677217100012, 'query_status': 'pass', 'start_time': 1747460114.2523649}, 'q2': {'query_name': 'q2', 'application_id': 'app-20250517053242-0003', 'application_time_taken': 17.682914708999306, 'query_status': 'pass', 'start_time': 1747460141.9399388}, 'q3': {'query_name': 'q3', 'application_id': 'app-20250517053242-0003', 'application_time_taken': 32.651489861998925, 'query_status': 'pass', 'start_time': 1747460159.6583893}, 'q4': {'query_name': 'q4', 'application_id': 'app-20250517053242-0003', 'application_time_taken': 24.433711993999168, 'query_status': 'pass', 'start_time': 1747460192.3545694}, 'q5': {'query_name': 'q5', 'application_id': 'app-20250517053242-0003', 'application_time_taken': 91.24168911599918, 'query_status': 'pass', 'start_time': 1747460216.8150887}, 'q6': {'query_name': 'q6', 'application_id': 'app-20250517053242-0003', 'application_time_taken': 8.312620381999295,

In [11]:
bm.collect_profile()

appid is  app-20250517053242-0003
saved to /opt/spark/work-dir/profile/


In [10]:
bm.sct.sc.stop()